In [142]:
import pandas as pd

In [143]:
df = pd.read_excel('../data_input/datasource.xlsx',sheet_name=0)

In [144]:
df_sample_plan = pd.read_excel('../data_input/Sampling Plan information.xlsx')

In [145]:
df_sample_plan.rename(columns={'Lot Size (from)':'Min','Lot Size (to)':'Max'},inplace=True)

In [146]:
df_sample_plan.loc[df_sample_plan.groupby(['Sampling Plan ID'])['Max'].idxmax(), 'Max'] = 99999999

In [148]:
df_normal = pd.read_excel('../data_input/datasource.xlsx',sheet_name=1)
df_reduce = pd.read_excel('../data_input/datasource.xlsx',sheet_name=2)

In [149]:
df_normal_Functional = df_normal.loc[df_normal['Criteria Type'] == 'Functional',['PIC','Sampling Plan ID']]
df_normal_Dimensional = df_normal.loc[df_normal['Criteria Type'] == 'Dimensional',['PIC','Sampling Plan ID']]
df_normal_Visual = df_normal.loc[df_normal['Criteria Type'] == 'Visual',['PIC','Sampling Plan ID']]

In [150]:
df_reduce_Functional = df_reduce.loc[df_reduce['Criteria Type'] == 'Functional',['PIC','Sampling Plan ID']]
df_reduce_Dimensional = df_reduce.loc[df_reduce['Criteria Type'] == 'Dimensional',['PIC','Sampling Plan ID']]
df_reduce_Visual = df_reduce.loc[df_reduce['Criteria Type'] == 'Visual',['PIC','Sampling Plan ID']]

In [151]:
df_reduce_Visual

,PIC,Sampling Plan ID
1,PIC-00001,188
2,PIC-00001,461
3,PIC-00001,189
4,PIC-00001,229
7,PIC-00002,189
...,...,...
653,Qual-99963,229
654,Qual-99963,241
655,Qual-99963,539
656,Qual-99963,267


In [152]:
def get_sample_size(row, df):
    # 在第三个数据框中查询符合条件的Sample Size
    sample_sizes = df[(df['Sampling Plan ID'] == row['Sampling Plan ID']) & 
                       (df['Min'] <= row['Qty EA']) & 
                       (df['Max'] >= row['Qty EA'])]['Sample Size']
    # 如果找到符合条件的Sample Size，则返回最大值
    return sample_sizes.max() if not sample_sizes.empty else None


In [153]:
df_base = (
    df
.assign(**{"Inspection Date" : lambda d : pd.to_datetime(d['Inspection Date'])})
.assign(key=lambda d: d.apply(lambda s:   
                            s['ID'] if s['Path'] == 'QIM'   
                            else '|'.join([str(s['Lot Number']),   
                                           s['Inspection Date'].date().strftime('%Y-%m-%d'),   
                                           str(s['Inspector']),   
                                           str(s['Item Number'])]),   
                          axis=1)
        )
.query('PIC != "Missing in ETQ"')
.groupby(['key','PIC'],as_index=False)['Qty EA'].sum()
)

In [154]:
df_base.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_2616\2162765692.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_base.sum()


PIC       PIC-00092PIC-00035PIC-00063PIC-00044PIC-00044P...
Qty EA                                         6517335215.0
dtype: object

In [155]:
df_sample_size_normal_Functional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_normal_Functional,on='PIC',how='left'))
    .assign(**{"Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_normal_Functional.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_2616\875044432.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_normal_Functional.sum()


PIC                 PIC-00035PIC-00035PIC-00035PIC-00035PIC-00035P...
Qty EA                                                   6517335215.0
Sampling Plan ID                                            1584147.0
Sample Size                                                  202089.0
dtype: object

In [156]:
df_sample_size_normal_Dimensional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_normal_Dimensional,on='PIC',how='left'))
    .assign(**{"Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_normal_Dimensional.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_2616\3591223376.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_normal_Dimensional.sum()


PIC                 PIC-00418PIC-00418PIC-00418PIC-00418PIC-00418P...
Qty EA                                                   6517335215.0
Sampling Plan ID                                            1440642.0
Sample Size                                                  235002.0
dtype: object

In [157]:
df_sample_size_normal_Visual = (
    df_base
    .pipe(lambda d : pd.merge(d,df_normal_Visual,on='PIC',how='left'))
    .assign(**{"Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_normal_Visual.sum()

C:\Users\ypeng\AppData\Local\Temp\ipykernel_2616\148947359.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sample_size_normal_Visual.sum()


PIC                 PIC-00058PIC-00058PIC-00058PIC-00058PIC-00058P...
Qty EA                                                   6517335215.0
Sampling Plan ID                                            1786566.0
Sample Size                                                 1903730.0
dtype: object

In [ ]:
df_sample_size_reduce_Functional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_reduce_Functional,on='PIC',how='left'))
    .assign(**{"Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_reduce_Functional.sum()

In [ ]:
df_sample_size_reduce_Dimensional = (
    df_base
    .pipe(lambda d : pd.merge(d,df_reduce_Dimensional,on='PIC',how='left'))
    .assign(**{"Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_reduce_Dimensional.sum()

In [ ]:
df_sample_size_reduce_Visual = (
    df_base
    .pipe(lambda d : pd.merge(d,df_reduce_Visual,on='PIC',how='left'))
    .assign(**{"Sample Size" : lambda d : d.apply(get_sample_size, axis=1, df=df_sample_plan)})
    .sort_values('Sample Size', ascending=False).drop_duplicates('key')
)
df_sample_size_reduce_Visual.sum()